In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)
# 데이터 출처 - https://www.nie-ecobank.kr/rdm/rsrchdoi/selectRsrchDtaListVw.do (에코뱅크(국제생태정보종합은행))

In [3]:
data_2014 = pd.read_csv("전국자연환경조사_조류_2014.csv")
data_2015 = pd.read_csv("전국자연환경조사_조류_2015.csv")
data_2016 = pd.read_csv("전국자연환경조사_조류_2016.csv")
data_2017 = pd.read_csv("전국자연환경조사_조류_2017.csv")
data_2018 = pd.read_csv("전국자연환경조사_조류_2018.csv")
data_2019 = pd.read_csv("전국자연환경조사_조류_2019.csv")

In [8]:
data_2014.isnull().sum()

id             0
학명             0
한글보통명          0
위도             0
경도             0
조사일자           0
조사연도           0
시도명           29
개체수         8428
geometry       0
dtype: int64

In [16]:
data_2014.columns

Index(['id', '학명', '한글보통명', '위도', '경도', '조사일자', '조사연도', '시도명', '개체수',
       'geometry'],
      dtype='object')

In [21]:
dup = data_2014.duplicated(data_2014.columns[1:])
df_dup = pd.concat([data_2014, dup], axis=1)
df_dup.rename(columns = {0 : 'Dup'}, inplace = True)
df_dup

,id,학명,한글보통명,위도,경도,조사일자,조사연도,시도명,개체수,geometry,Dup
0,8000000002364164,Podiceps cristatus,뿔논병아리,36.733111,126.654500,2014-03-14,2014,충청남도,NaN,POINT (169139.2620999999 459450.5734999999),False
1,8000000002364165,Podiceps cristatus,뿔논병아리,36.733111,126.654500,2014-03-14,2014,충청남도,NaN,POINT (169139.2620999999 459450.5734999999),True
2,8000000002097468,Dendrocopos major,오색딱다구리,36.655028,126.638667,2014-03-15,2014,충청남도,NaN,POINT (167692.3596000001 450790.7155000009),False
3,8000000002097453,Buteo buteo,말똥가리,36.640000,126.694306,2014-03-15,2014,충청남도,NaN,POINT (172661.8948999997 449105.7410000004),False
4,8000000002097451,Picus canus,청딱다구리,36.636556,126.686917,2014-03-15,2014,충청남도,NaN,POINT (171999.8331000004 448725.6973000001),False
...,...,...,...,...,...,...,...,...,...,...,...
9388,8000000002276849,Dendrocopos major,오색딱다구리,37.953536,127.046392,2014-10-30,2014,경기도,NaN,POINT (204077.2844000002 594843.7075999994),False
9389,8000000002278839,Dendrocopos major,오색딱다구리,35.653889,129.073944,2014-10-30,2014,울산광역시,NaN,POINT (387821.3569 341623.5326000005),False
9390,8000000002301998,Buteo buteo,말똥가리,37.950919,127.122331,2014-10-31,2014,경기도,NaN,POINT (210751.8006999996 594559.2513999995),False
9391,8000000002302007,Dendrocopos major,오색딱다구리,37.973867,127.072175,2014-10-31,2014,경기도,NaN,POINT (206341.5571999997 597101.7865999993),False


In [23]:
df_dup[df_dup['Dup']==True]['개체수'].unique()

array([nan,  1.])

In [11]:
data_2014.drop_duplicates()

,id,학명,한글보통명,위도,경도,조사일자,조사연도,시도명,개체수,geometry
0,8000000002364164,Podiceps cristatus,뿔논병아리,36.733111,126.654500,2014-03-14,2014,충청남도,NaN,POINT (169139.2620999999 459450.5734999999)
1,8000000002364165,Podiceps cristatus,뿔논병아리,36.733111,126.654500,2014-03-14,2014,충청남도,NaN,POINT (169139.2620999999 459450.5734999999)
2,8000000002097468,Dendrocopos major,오색딱다구리,36.655028,126.638667,2014-03-15,2014,충청남도,NaN,POINT (167692.3596000001 450790.7155000009)
3,8000000002097453,Buteo buteo,말똥가리,36.640000,126.694306,2014-03-15,2014,충청남도,NaN,POINT (172661.8948999997 449105.7410000004)
4,8000000002097451,Picus canus,청딱다구리,36.636556,126.686917,2014-03-15,2014,충청남도,NaN,POINT (171999.8331000004 448725.6973000001)
...,...,...,...,...,...,...,...,...,...,...
9388,8000000002276849,Dendrocopos major,오색딱다구리,37.953536,127.046392,2014-10-30,2014,경기도,NaN,POINT (204077.2844000002 594843.7075999994)
9389,8000000002278839,Dendrocopos major,오색딱다구리,35.653889,129.073944,2014-10-30,2014,울산광역시,NaN,POINT (387821.3569 341623.5326000005)
9390,8000000002301998,Buteo buteo,말똥가리,37.950919,127.122331,2014-10-31,2014,경기도,NaN,POINT (210751.8006999996 594559.2513999995)
9391,8000000002302007,Dendrocopos major,오색딱다구리,37.973867,127.072175,2014-10-31,2014,경기도,NaN,POINT (206341.5571999997 597101.7865999993)


In [10]:
data_2014[data_2014['시도명'].isna()]

,id,학명,한글보통명,위도,경도,조사일자,조사연도,시도명,개체수,geometry
148,8000000002110692,Larus crassirostris,괭이갈매기,37.816917,126.194972,2014-04-06,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
150,8000000002110693,Larus crassirostris,괭이갈매기,37.816917,126.194972,2014-04-06,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
156,8000000002110690,Phalacrocorax capillatus,가마우지,37.816917,126.194972,2014-04-06,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
160,8000000002110691,Phalacrocorax capillatus,가마우지,37.816917,126.194972,2014-04-06,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
1530,8000000002110700,Ardea alba modesta,중대백로,37.816917,126.194972,2014-05-10,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
1532,8000000002110701,Ardea alba modesta,중대백로,37.816917,126.194972,2014-05-10,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
1581,8000000002110698,Phalacrocorax capillatus,가마우지,37.816917,126.194972,2014-05-10,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
1585,8000000002110699,Phalacrocorax capillatus,가마우지,37.816917,126.194972,2014-05-10,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
1642,8000000002110696,Larus crassirostris,괭이갈매기,37.816917,126.194972,2014-05-10,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)
1648,8000000002110697,Larus crassirostris,괭이갈매기,37.816917,126.194972,2014-05-10,2014,NaN,NaN,POINT (129116.3912000004 579984.0800999999)


In [4]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], tiles='cartodbpositron', zoom_start=12)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 통합 도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2014 = folium.FeatureGroup(name="2014년", tiles='cartodbpositron', overlay=False)
m_2015 = folium.FeatureGroup(name="2015년", tiles='cartodbpositron', overlay=False)
m_2016 = folium.FeatureGroup(name="2016년", tiles='cartodbpositron', overlay=False)
m_2017 = folium.FeatureGroup(name="2017년", tiles='cartodbpositron', overlay=False)
m_2018 = folium.FeatureGroup(name="2018년", tiles='cartodbpositron', overlay=False)
m_2019 = folium.FeatureGroup(name="2019년", tiles='cartodbpositron', overlay=False)

year_lst = [str(year) for year in range(2014, 2020)]
data_lst = [data_2014, data_2015, data_2016, data_2017, data_2018, data_2019]
layers_lst = [m_2014, m_2015, m_2016, m_2017, m_2018, m_2019]
data_dic = {}
layers_dic = {}
for year, data, layer in zip(year_lst, data_lst, layers_lst):
    data_dic[year] = data
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2018부터 2022까지의 연도
    # GeoDataFrame을 순회하면서 Polygon을 지도에 추가
    for idx, row in data_dic[year].iterrows():
        marker = folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=1,
            fill=True,
            color="Red",
            fillColor="Red"
        ).add_to(layers_dic[year])#.add_child(folium.Popup(popup_text, max_width=100))

for layer_instance in layers_dic.values():
    layer_instance.add_to(m)

# LayerControl을 사용하여 주중 & 주말 선택할 수 있게 구성
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html
m.save("연도별_조류분포_시각화.html")

In [19]:
m.save("연도별_조류분포_시각화.html")

In [ ]:
shp_file_path = "백두대간 정밀조사_조류_2013_2019_전국/백두대간 정밀조사_조류_2013_2019_전국.shp"
# shp 파일을 GeoDataFrame으로 읽기
gdf = gpd.read_file(shp_file_path)

In [ ]:
# https://sgis.kostat.go.kr/view/pss/requstDataListDtl 통계지리정보 서비스에 자료신청함(2024-01-12)
# https://egis.me.go.kr/ 여기 데이터 찾아보기